In [57]:
%pip install tabulate


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#REPORT MODULE SINGLE CALCULATION

In [67]:
import requests
import pandas as pd
import json

# Define the input data for the CO2 report with optional vessel IMO and SCAC
shipping_details = [
    {
        "from_locode": "CNSHA",
        "to_locode": "NLRTM",
        "n_containers": 1,
        "weight": 10000,
        "carrier_scac": "CMDU",  # Optional SCAC
        "vessel_imo": "9893917",  # Optional vessel IMO
        "departure_date": "2024-01-15T00:00:00Z"  # Example departure date
    },
    # Add more shipping details as needed
]

# Define the API key
api_key = "NhOS9QL56mvfzBtCZlmU72LbESzbhSh2LQ4VxU5k"

# Initialize a list to collect final report data
final_reports = []

# Define headers for the API request
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "x-api-key": api_key
}

# Iterate over the shipping details and generate CO2 reports
for details in shipping_details:
    # Create the payload for the final call
    payload = {
        "orders": [
            {
                "type": "CONTAINER",
                "quantity": details["n_containers"],
                "weight": details["weight"]
            }
        ],
        "legs": [
            {
                "from": details["from_locode"],
                "to": details["to_locode"],
                "mode": "sea",
                "details": {
                    "dateTime": {"departure": details["departure_date"]}
                }
            }
        ]
    }

    # Add carrier SCAC to the payload if it's provided
    if details.get("carrier_scac"):
        payload["legs"][0]["details"]["carrier"] = {"scac": details["carrier_scac"]}

    # Add vessel IMO to the payload if it's provided
    if details.get("vessel_imo"):
        payload["legs"][0]["details"]["vessel"] = {"imo": str(details["vessel_imo"])}
    
    # Make the API call to get the final report
    try:
        final_url = "https://api.searoutes.com/shipment/v2/report/co2?generateCertificate=false"
        final_response = requests.post(final_url, json=payload, headers=headers, verify=False)
        final_response.raise_for_status()  # Check for HTTP errors
        final_report = final_response.json()

        # Add additional details to the final report
        final_report.update(details)

        # Add the final report to the list
        final_reports.append(final_report)
    except requests.exceptions.RequestException as e:
        # Print detailed error information
        print(f"An error occurred: {e}")
        if e.response is not None:
            print(f"Response content: {e.response.content}")

# Convert the final reports to a DataFrame
df_reports = pd.json_normalize(final_reports)

# Write the DataFrame to Excel
df_reports.to_excel("final_report_only.xlsx", sheet_name='Final Reports', index=False)

# Print the final reports
print("\nFinal Reports:")
for report in final_reports:
    print(json.dumps(report, indent=4))

print("Final reports exported to final_report_only.xlsx")


c:\Users\wsealho\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.searoutes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Final Reports:
{
    "parameters": {
        "orders": [
            {
                "type": "CONTAINER",
                "quantity": 1,
                "weight": 10000
            }
        ],
        "details": {}
    },
    "co2e": {
        "total": 636496,
        "wtt": 48527,
        "ttw": 587968,
        "intensity": 0.00241
    },
    "legs": [
        {
            "from": {
                "locode": "CNSHA",
                "coordinates": [
                    122.06550521343334,
                    30.61798396262171
                ],
                "city": "Shanghai",
                "country": "China",
                "region": "North East Asia"
            },
            "to": {
                "locode": "NLRTM",
                "coordinates": [
                    4.056346273311497,
                    51.96013439389793
                ],
                "city": "Rotterdam",
                "country": "Netherlands",
                "region": "North Europe"
        

In [51]:
#PLAN

In [71]:
import requests
import pandas as pd

# Define input variables for itinerary search
from_locode = "CNSHA"
to_locode = "NLRTM"
carrier_scac = "CMDU"
n_containers = 1
weight = 10000
api_key = "NhOS9QL56mvfzBtCZlmU72LbESzbhSh2LQ4VxU5k"
from_date = "2024-05-16T00:00:00Z"  # ISO 8601 format with time
to_date = "2024-05-30T23:59:59Z"  # ISO 8601 format with time
sort_by = "CO2"  # Can be "CO2" or "TRANSIT_TIME"

# Print input variables
print("Input variables for itinerary search:")
print(f"from_locode: {from_locode}")
print(f"to_locode: {to_locode}")
print(f"carrier_scac: {carrier_scac}")
print(f"n_containers: {n_containers}")
print(f"weight: {weight}")
print(f"from_date: {from_date}")
print(f"to_date: {to_date}")
print(f"sort_by: {sort_by}")

# Create the URL for itinerary search
itinerary_url = (f"https://api.searoutes.com/itinerary/v2/execution?fromLocode={from_locode}"
                 f"&toLocode={to_locode}&carrierScac={carrier_scac}&nContainers={n_containers}"
                 f"&weight={weight}&fromDate={from_date}&toDate={to_date}&sortBy={sort_by}")

# Define headers
headers = {
    "accept": "application/json",
    "x-api-key": api_key
}

# Make the API call to get itineraries
try:
    itinerary_response = requests.get(itinerary_url, headers=headers, verify=False)
    itinerary_response.raise_for_status()  # Check for HTTP errors
    itineraries = itinerary_response.json()

    # Print itineraries for debugging
    print("Itineraries response:")
    print(itineraries)

    # Initialize a list to collect CO2 details
    all_options = []
    best_option = None

    # Extract the hash from each itinerary and get CO2 details
    if itineraries and isinstance(itineraries, list) and len(itineraries) > 0:
        for itinerary in itineraries:
            if 'properties' in itinerary and 'hash' in itinerary['properties']:
                itinerary_hash = itinerary['properties']['hash']
                print(f"Processing CO2 details for itinerary with hash: {itinerary_hash}")

                # Create the URL for CO2 details
                co2_url = f"https://api.searoutes.com/co2/v2/execution/{itinerary_hash}"

                # Make the API call to get CO2 details
                co2_response = requests.get(co2_url, headers=headers, verify=False)
                co2_response.raise_for_status()  # Check for HTTP errors
                co2_details = co2_response.json()

                # Add additional details to the CO2 details
                co2_details.update({
                    'itinerary_hash': itinerary_hash,
                    'vessel_imo': itinerary['features'][0]['properties']['asset']['imo'],
                    'vessel_name': itinerary['features'][0]['properties']['asset']['name'],
                    'vessel_min_teu': itinerary['features'][0]['properties']['asset']['minTEU'],
                    'vessel_max_teu': itinerary['features'][0]['properties']['asset']['maxTEU'],
                    'vessel_fuel_type': itinerary['features'][0]['properties']['asset']['fuelType'],
                    'departure_date': itinerary['features'][0]['properties']['departure']['dateTime'],
                    'arrival_date': itinerary['features'][0]['properties']['arrival']['dateTime'],
                    'distance': itinerary['features'][0]['properties']['distance'],
                    'mode': itinerary['features'][0]['properties']['mode']
                })

                # Add the CO2 details to the list of all options
                all_options.append(co2_details)

                # Determine if this is the best option (least CO2 emissions)
                if not best_option or co2_details['co2e']['total'] < best_option['co2e']['total']:
                    best_option = co2_details
            else:
                print("No valid itinerary hash found.")
    else:
        print("No itineraries found.")

    # Print all CO2 options
    print("\nAll CO2 options:")
    for option in all_options:
        print(option)

    # Print the best CO2 option
    if best_option:
        print("\nBest CO2 option:")
        print(best_option)
    else:
        print("No valid CO2 details found.")

except requests.exceptions.RequestException as e:
    # Print detailed error information
    print(f"An error occurred: {e}")
    if e.response is not None:
        print(f"Response content: {e.response.content}")


Input variables for itinerary search:
from_locode: CNSHA
to_locode: NLRTM
carrier_scac: CMDU
n_containers: 1
weight: 10000
from_date: 2024-05-16T00:00:00Z
to_date: 2024-05-30T23:59:59Z
sort_by: CO2


c:\Users\wsealho\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.searoutes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Itineraries response:
[{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'duration': {'ms': 3477600000}, 'departure': {'dateTime': '2024-05-19T00:00:00Z', 'timestamp': 1716076800000}, 'arrival': {'dateTime': '2024-06-28T06:00:00Z', 'timestamp': 1719554400000}, 'from': {'locode': 'CNSHA', 'name': 'Shanghai'}, 'to': {'locode': 'NLRTM', 'name': 'Rotterdam'}, 'distance': 19825749, 'mode': 'SEA', 'serviceId': 1961, 'asset': {'imo': 9839181, 'name': 'CMA CGM Palais Royal', 'minTEU': 20000, 'maxTEU': 999999, 'fuelType': 'LNG'}, 'calls': [{'locode': 'CNSHA', 'name': 'Shanghai'}, {'locode': 'CNNGB', 'name': 'Ningbo'}, {'locode': 'CNYTN', 'name': 'Yantian Pt'}, {'locode': 'SGSIN', 'name': 'Singapore'}, {'locode': 'FRDKK', 'name': 'Dunkerque'}, {'locode': 'NLRTM', 'name': 'Rotterdam'}], 'co2e': 556864}, 'geometry': {'type': 'LineString', 'coordinates': []}}], 'properties': {'hash': '4nuQcWNl5h5'}}, {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'propert

c:\Users\wsealho\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.searoutes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Processing CO2 details for itinerary with hash: GsB47OgLRW1


...

c:\Users\wsealho\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.searoutes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Processing CO2 details for itinerary with hash: Bos6G2DrGCq


c:\Users\wsealho\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.searoutes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Processing CO2 details for itinerary with hash: 1X3nnN6FvlB

All CO2 options:
{'co2e': {'total': 556864, 'wtt': 151416, 'ttw': 405448, 'intensity': 0.00281}, 'parameters': {'mode': 'SEA', 'carrierId': None, 'carrierScac': 'CMDU', 'from': {'locode': 'CNSHA'}, 'to': {'locode': 'NLRTM'}, 'containerSizeTypeCode': None, 'weight': 10000.0, 'nContainers': 1.0}, 'properties': {'legs': [{'co2e': {'total': 556864, 'wtt': 151416, 'ttw': 405448, 'intensity': 0.00281}, 'properties': {'dataType': None, 'distance': 19825749, 'secaIntersection': 0, 'asset': {'imo': 9839181, 'name': 'CMA CGM Palais Royal', 'minTEU': 20000, 'maxTEU': 999999, 'fuelType': 'LNG'}, 'tradeLane': None}}], 'weight': 10000.0, 'nTEU': 1.0, 'containerSizeTypeCode': '20GP'}, 'itinerary_hash': '4nuQcWNl5h5', 'vessel_imo': 9839181, 'vessel_name': 'CMA CGM Palais Royal', 'vessel_min_teu': 20000, 'vessel_max_teu': 999999, 'vessel_fuel_type': 'LNG', 'departure_date': '2024-05-19T00:00:00Z', 'arrival_date': '2024-06-28T06:00:00Z', 'dist

c:\Users\wsealho\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.searoutes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
#REPORT

In [73]:
# Initialize a list to collect final report data
final_reports = []

# Extract the necessary details and run the final report for each option
for option in all_options:
    from_locode = option['parameters']['from']['locode']
    to_locode = option['parameters']['to']['locode']
    n_containers = option['parameters']['nContainers']
    weight = option['parameters']['weight']
    vessel_imo = str(option['vessel_imo'])  # Ensure IMO is a string
    departure_date = option['departure_date']  # Get departure date

    # Create the payload for the final call
    payload = {
        "orders": [
            {
                "type": "CONTAINER",
                "quantity": n_containers,
                "weight": weight
            }
        ],
        "legs": [
            {
                "from": from_locode,
                "to": to_locode,
                "mode": "sea",
                "details": {
                    "carrier": {"scac": carrier_scac},
                    "dateTime": {"departure": departure_date},
                    "vessel": {"imo": vessel_imo}
                }
            }
        ]
    }

    # Define headers
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "x-api-key": api_key
    }

    # Make the API call to get the final report
    try:
        final_url = "https://api.searoutes.com/shipment/v2/report/co2?generateCertificate=false"
        final_response = requests.post(final_url, json=payload, headers=headers, verify=False)
        final_response.raise_for_status()  # Check for HTTP errors
        final_report = final_response.json()

        # Add additional details to the final report
        final_report.update({
            'itinerary_hash': option['itinerary_hash'],
            'vessel_imo': option['vessel_imo'],
            'vessel_name': option['vessel_name'],
            'vessel_min_teu': option['vessel_min_teu'],
            'vessel_max_teu': option['vessel_max_teu'],
            'vessel_fuel_type': option['vessel_fuel_type'],
            'departure_date': option['departure_date'],
            'arrival_date': option['arrival_date'],
            'distance': option['distance'],
            'mode': option['mode']
        })

        # Print each final report
        print("\nFinal Report:")
        print(json.dumps(final_report, indent=4))

        # Add the final report to the list
        final_reports.append(final_report)
    except requests.exceptions.RequestException as e:
        # Print detailed error information
        print(f"An error occurred: {e}")
        if e.response is not None:
            print(f"Response content: {e.response.content}")

# Convert the CO2 details to a DataFrame
df_options = pd.json_normalize(all_options)
df_reports = pd.json_normalize(final_reports)

# Write the DataFrames to Excel
with pd.ExcelWriter("final_reports.xlsx") as writer:
    df_options.to_excel(writer, sheet_name='CO2 Options', index=False)
    df_reports.to_excel(writer, sheet_name='Final Reports', index=False)

print("Final reports exported to final_reports.xlsx")

c:\Users\wsealho\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.searoutes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Final Report:
{
    "parameters": {
        "orders": [
            {
                "type": "CONTAINER",
                "quantity": 1,
                "weight": 10000
            }
        ],
        "details": {}
    },
    "co2e": {
        "total": 631836,
        "wtt": 171803,
        "ttw": 460033,
        "intensity": 0.00323
    },
    "legs": [
        {
            "from": {
                "locode": "CNSHA",
                "coordinates": [
                    122.06550521343334,
                    30.61798396262171
                ],
                "city": "Shanghai",
                "country": "China",
                "region": "North East Asia"
            },
            "to": {
                "locode": "NLRTM",
                "coordinates": [
                    4.056346273311497,
                    51.96013439389793
                ],
                "city": "Rotterdam",
                "country": "Netherlands",
                "region": "North Europe"
        

c:\Users\wsealho\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.searoutes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Final Report:
{
    "parameters": {
        "orders": [
            {
                "type": "CONTAINER",
                "quantity": 1,
                "weight": 10000
            }
        ],
        "details": {}
    },
    "co2e": {
        "total": 685868,
        "wtt": 101352,
        "ttw": 584516,
        "intensity": 0.00351
    },
    "legs": [
        {
            "from": {
                "locode": "CNSHA",
                "coordinates": [
                    122.06550521343334,
                    30.61798396262171
                ],
                "city": "Shanghai",
                "country": "China",
                "region": "North East Asia"
            },
            "to": {
                "locode": "NLRTM",
                "coordinates": [
                    4.056346273311497,
                    51.96013439389793
                ],
                "city": "Rotterdam",
                "country": "Netherlands",
                "region": "North Europe"
        

c:\Users\wsealho\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.searoutes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...


Final Report:
{
    "parameters": {
        "orders": [
            {
                "type": "CONTAINER",
                "quantity": 1,
                "weight": 10000
            }
        ],
        "details": {}
    },
    "co2e": {
        "total": 734711,
        "wtt": 56019,
        "ttw": 678692,
        "intensity": 0.00376
    },
    "legs": [
        {
            "from": {
                "locode": "CNSHA",
                "coordinates": [
                    122.06550521343334,
                    30.61798396262171
                ],
                "city": "Shanghai",
                "country": "China",
                "region": "North East Asia"
            },
            "to": {
                "locode": "NLRTM",
                "coordinates": [
                    4.056346273311497,
                    51.96013439389793
                ],
                "city": "Rotterdam",
                "country": "Netherlands",
                "region": "North Europe"
         

c:\Users\wsealho\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.searoutes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Final Report:
{
    "parameters": {
        "orders": [
            {
                "type": "CONTAINER",
                "quantity": 1,
                "weight": 10000
            }
        ],
        "details": {}
    },
    "co2e": {
        "total": 706688,
        "wtt": 104428,
        "ttw": 602260,
        "intensity": 0.00361
    },
    "legs": [
        {
            "from": {
                "locode": "CNSHA",
                "coordinates": [
                    122.06550521343334,
                    30.61798396262171
                ],
                "city": "Shanghai",
                "country": "China",
                "region": "North East Asia"
            },
            "to": {
                "locode": "NLRTM",
                "coordinates": [
                    4.056346273311497,
                    51.96013439389793
                ],
                "city": "Rotterdam",
                "country": "Netherlands",
                "region": "North Europe"
        

c:\Users\wsealho\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.searoutes.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



Final Report:
{
    "parameters": {
        "orders": [
            {
                "type": "CONTAINER",
                "quantity": 1,
                "weight": 10000
            }
        ],
        "details": {}
    },
    "co2e": {
        "total": 734711,
        "wtt": 56019,
        "ttw": 678692,
        "intensity": 0.00376
    },
    "legs": [
        {
            "from": {
                "locode": "CNSHA",
                "coordinates": [
                    122.06550521343334,
                    30.61798396262171
                ],
                "city": "Shanghai",
                "country": "China",
                "region": "North East Asia"
            },
            "to": {
                "locode": "NLRTM",
                "coordinates": [
                    4.056346273311497,
                    51.96013439389793
                ],
                "city": "Rotterdam",
                "country": "Netherlands",
                "region": "North Europe"
         